In [10]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Generate_summaries_research_with_agents"

In [11]:
%pwd

'c:\\Users\\Maza\\Desktop\\Pinecone_pipeline\\research'

In [12]:
os.chdir("../")

In [13]:
%pwd

'c:\\Users\\Maza\\Desktop\\Pinecone_pipeline'

In [14]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class JsonSummaryConfig:
    """
    Configuration class for JSON summary settings.
    
    This class holds configuration settings required for handling JSON summaries.
    It includes paths for directories and files as well as dictionaries for prompt 
    generation and model settings.

    Attributes:
        root_dir (Path): The root directory where all related files and directories are located.
        read_schema (Path): The path to the schema file used for reading or validating JSON data.
        load_json_summary (Path): The path where the JSON summary will be loaded from or saved to.
        prompt_generate_json_summary (dict): A dictionary containing prompts used for generating JSON summaries.
        models (dict): A dictionary containing model configurations for generating summaries.
    """
    root_dir: Path
    read_schema:Path
    load_json_summary: Path
    prompt_generate_json_summary:dict
    models: dict

In [15]:
from langgraph.graph import END, StateGraph
from typing_extensions import TypedDict
@dataclass(frozen=True)
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    This TypedDict class defines the structure for storing the state of a graph
    during its processing. It includes details about the input file, intermediate
    summaries, feedback, final summary, and other relevant metadata.

    Attributes:
        initial_file (str): The initial file path or name used as input for the graph.
        draft_json_summary (dict): A dictionary holding the draft version of the JSON summary.
        json_feedback (dict): A dictionary holding feedback for the JSON summary.
        final_json_summary (dict): A dictionary holding the final version of the JSON summary.
        num_steps (int): The number of steps or iterations taken in the graph processing.
       
    """
    initial_file : str
    draft_json_summary : dict
    json_feedback : dict
    final_json_summary : dict
    num_steps : int



In [25]:
from vector_db_pipeline.utils.common import load_json, read_yaml, create_directories,save_json, read_txt_to_list
from pathlib import Path
from langchain_core.messages import  HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from vector_db_pipeline.constants import *
from vector_db_pipeline import logger
import json
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser

load_dotenv()

True

In [17]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        models_filepath = MODELS_FILE_PATH,
        prompt_template = PROMPT_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        self.params = read_yaml(params_filepath)
        self.models = read_yaml(models_filepath)
        self.prompt_template = read_yaml(prompt_template)
    
        

    
    def get_json_summary_config(self) -> JsonSummaryConfig:
        """
        Retrieves the configuration settings for JSON summary processing.

        This method reads the configuration settings from the class instance's config attribute,
        creates necessary directories, and initializes a JsonSummaryConfig object with the 
        retrieved settings.

        Returns:
            JsonSummaryConfig: An object containing configuration settings for JSON summary processing.
        """
        config = self.config.json_summary
        create_directories([config.root_dir])
        
        json_summary_config = JsonSummaryConfig(
            root_dir=config.root_dir,
            read_schema = config.read_schema,
            load_json_summary = config.load_json_summary,
            prompt_generate_json_summary = self.prompt_template.generate_json_summary,
            models = self.models,
            
        ) 

        return json_summary_config

In [87]:
def read_file_with_encodings(file_path):
    """
    Tries to read a file with different encodings until successful.

    Args:
        file_path (Path): The path to the file to be read.

    Returns:
        str: The content of the file if read successfully, otherwise an empty string.
    """
    encodings = ['utf-8', 'latin-1', 'iso-8859-1', 'windows-1252']
    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as f:
                content = f.read()
            logger.info(f"File content successfully read using encoding: {encoding}")
            return content
        except UnicodeDecodeError:
            logger.warning(f"Failed to read file with encoding: {encoding}")
    return ""

In [91]:
class JsonSummary:
    """
    A class to generate and manage JSON summaries using various agents and models.

    Attributes:
        config (JsonSummaryConfig): Configuration settings for JSON summary processing.
    """
    def __init__(self, config:JsonSummaryConfig):
        """
        Initializes the JsonSummary class with the provided configuration.

        Args:
            config (JsonSummaryConfig): Configuration settings for JSON summary processing.
        """
        self.config = config

   
    
    def configure_model_system(self):
        """
        Configures the model system with the appropriate agents and prompts.

        This method sets up the models, templates, and agents required for generating,
        validating, and editing JSON summaries.
        """
        model =self.config.models.Llama3
        logger.info(f"Working with model: {model}")
        llm = ChatGroq(temperature=0, model_name=model)
        
        #Agents

        # summary generator agent 
        json_creator_promt = PromptTemplate(
            template= self.config.prompt_generate_json_summary.agent_summary_json_creator,
            input_variables=["content"])
        
        self.json_summary_generator = json_creator_promt | llm | JsonOutputParser()
        
        # analyse summary and decides if it has correct format
        data_type_json_route_prompt = PromptTemplate(
            template= self.config.prompt_generate_json_summary.agent_edit_json_route,
            input_variables=["file"])
        
        self.data_type_json_route_generator = data_type_json_route_prompt | llm | StrOutputParser()

        # if json format is not correct this agent provides feedback for the final editor to aply corrections
        json_feedbak_prompt =  PromptTemplate(
            template= self.config.prompt_generate_json_summary.agent_feedbak_json,
            input_variables=["file"])
        
        self.json_feedbak_generator = json_feedbak_prompt | llm | JsonOutputParser()


        # Final agent produce the final json object using the draft from self.json_summary_generator
        # and the feedback from json_feedbak_generator
        json_editor_prompt =  PromptTemplate(
            template= self.config.prompt_generate_json_summary.agent_rewrite_json,
            input_variables=["file", "feedback"])
        self.json_editor_generator = json_editor_prompt | llm | JsonOutputParser()
        
        
    
    
    def generate_json_summary_agent(self,state):
        """
        Generates a JSON summary draft from the initial file.

        Args:
            state (dict): The current state of the processing, including the initial file and step count.

        Returns:
            dict: Updated state with the draft JSON summary and incremented step count.
        """
        logger.info(f"Creating JSON summary draft")
        initial_file= state['initial_file']
        num_steps = int(state['num_steps'])
        num_steps += 1

        try:
            draft_json_summary = self.json_summary_generator.invoke({"content": initial_file})
        except Exception as e:
            logger.error(f"Error in json summary generator agent: {e}")
        return {"draft_json_summary": draft_json_summary, "num_steps":num_steps}
    
    def json_format_route_agent(self, state):
        """
        Determines if the JSON draft has the correct format.

        Args:
            state (dict): The current state of the processing, including the draft JSON summary and step count.

        Returns:
            str: 'edit_file' if the data type is incorrect, otherwise 'no_edit'.
        """
        logger.info(f"Checking JSON draft data type")
        draft_json_summary = state.get('draft_json_summary')
    
        if draft_json_summary is None:
            logger.error("No draft JSON summary provided.")
            return None

        try:
            summary_type = self.data_type_json_route_generator.invoke({"file": draft_json_summary})
        except Exception as e:
            logger.error(f"Error determining JSON draft data type: {e}")
            return None

        if summary_type == 'text':
            return 'edit_file'
        else:
            return 'no_edit'
        
    def feedback_json(self,state):
        """
        Provides feedback for correcting the JSON draft if the format is incorrect.

        Args:
            state (dict): The current state of the processing, including the draft JSON summary and step count.

        Returns:
            dict: Updated state with the feedback for the JSON draft and incremented step count.
        """

        logger.info(f"JSON draft incorrect.... Producing feedback ")
        draft_json_summary= state['draft_json_summary']
        num_steps = int(state['num_steps'])
        num_steps += 1
        try:
            json_feedback = self.json_feedbak_generator.invoke({"draft_json": draft_json_summary})
        except Exception as e:
            logger.error(f"Error producing fedback: {e}")
      
        return {"json_feedback": json_feedback, "num_steps":num_steps}

    
    def no_rewrite(self,state):
        """
        Finalizes the JSON draft as the final JSON summary if no corrections are needed.

        Args:
            state (dict): The current state of the processing, including the draft JSON summary and step count.

        Returns:
            dict: Updated state with the final JSON summary and incremented step count.
        """

        logger.info("JSON summary draft correct and assigned to final_json_summary")
        ## Get the state
        draft_json_summary = state["draft_json_summary"]
        num_steps = state['num_steps']
        num_steps += 1

        return {"final_json_summary": draft_json_summary, "num_steps":num_steps}
    
    

    def edit_file_agent(self,state):
        """
        Produces the final JSON summary using the draft and feedback.

        Args:
            state (dict): The current state of the processing, including the draft JSON summary, feedback, and step count.

        Returns:
            dict: Updated state with the final JSON summary and incremented step count.
        """
        
        logger.info(f"Acting on feedback and producing final JSON summary")
        draft_json_summary= state['draft_json_summary']
        json_feedback = state['json_feedback']
        num_steps = int(state['num_steps'])
        num_steps += 1
        
        try:
            final_json_summary = self.json_editor_generator.invoke({"file": draft_json_summary, "feedback":json_feedback })
        except Exception as e:
            logger.error(f"Error acting on fedback to generate final json file: {e}")


        return {"final_json_summary": final_json_summary, "num_steps":num_steps}
       
    def state_printer(self,state):
        """
        Prints the current state of the processing.

        Args:
            state (dict): The current state of the processing.
        """
        if state['num_steps'] > 2 :
            logger.info("---STATE PRINTER---")
            logger.info(f"Draft Json Summary Type: {type(state['draft_json_summary'])} \n")
            logger.info(f"Feedback: {state['json_feedback']} \n")
            logger.info(f"Final Json Summary Type: {type(state['final_json_summary'])} \n" )
            logger.info(f"Num Steps: {state['num_steps']} \n")
        else:
            logger.info("---STATE PRINTER---")
            logger.info(f"Draft Json Summary: {type(state['draft_json_summary'])} \n")
            logger.info(f"Final Json Summary: {type(state['final_json_summary'])} \n")
            logger.info(f"Num Steps: {state['num_steps']} \n")
    
    
    
    def create_graph_agents(self, workflow):
        """
        Creates and configures the agent workflow graph.

        Args:
            workflow (Workflow): The workflow object to which nodes and edges will be added.

        Returns:
            CompiledWorkflow: The compiled workflow ready for execution.
        """


        #nodes
        workflow.add_node("generate_json_summary_agent", self.generate_json_summary_agent) 
        workflow.add_node("feedback_json", self.feedback_json)
        workflow.add_node("edit_file_agent", self.edit_file_agent)
        workflow.add_node("no_rewrite", self.no_rewrite)
        workflow.add_node("state_printer", self.state_printer)

        #edges
        workflow.set_entry_point("generate_json_summary_agent")
        workflow.add_conditional_edges(
            "generate_json_summary_agent",
            self.json_format_route_agent,
            {
                "edit_file": "feedback_json",
                "no_edit": "no_rewrite",
            },
        )
        workflow.add_edge("feedback_json", "edit_file_agent")
        workflow.add_edge("no_rewrite", "state_printer")
        workflow.add_edge("edit_file_agent", "state_printer")
        workflow.add_edge("state_printer", END)

        #compile
        try:
            app = workflow.compile()
            logger.info("Agent graph created")
        except Exception as e:
            logger.error(f"Agent graph error: {e}")
             
        return  app
    
    def run_graph_agents(self, app,file_paths):
        """
        Runs the agent workflow on a list of file paths to generate JSON summaries.

        Args:
            app (CompiledWorkflow): The compiled workflow to be executed.
            file_paths (list[Path]): List of file paths to process.

        Returns:
            None
        """
        load_json_file = self.config.load_json_summary
        logger.info(f"Initializing agent summary orquestration")
        batch_size = 5


        batches = [file_paths[i:i + batch_size] for i in range(0, len(file_paths), batch_size)]
        
        for idx, files_batch in enumerate(batches, 1):

            if os.path.exists(Path(load_json_file)):
                json_summaries = load_json(Path(load_json_file))
                logger.info(f"Json summaries readed")
            else:
                json_summaries = {}
                logger.info(f"new Json summaries")
            
            
            logger.info(f"------------------------------Batch: {idx}------------------------------")
            
            for file_path in files_batch:
                logger.info(f"Starting summary of {file_path}")
                file_content = read_file_with_encodings(file_path)

                if len(file_content)>0:
                    filename = os.path.basename(file_path)
                    json_summaries[filename] = {}
                    json_summaries[filename]['FILE_PATH'] = str(file_path)
                    
                    inputs = {"initial_file": file_content,"num_steps":0}
                    try:
                        output = app.invoke(inputs)
                        json_summaries[filename].update(output['final_json_summary'])
                        logger.info(f"{filename} summary success")
                
                    except:
                        logger.error(f"Error in agent graph: {e}")
                   
                else:
                    logger.info(f"{filename} empty.")

            try:
                save_json(Path(load_json_file), json_summaries)
                logger.info(f"JSON summaries loaded to {load_json_file}")
            except Exception as e:
                logger.error(f"Error saving JSON summaries: {e}")
                return batches[idx:]
        
        return logger.info(f"All batches proccessed")



    

In [92]:
import time

In [93]:
start = time.time()

json_summary_config = ConfigurationManager()
config_json = json_summary_config.get_json_summary_config()
files_in_app = read_txt_to_list(Path(config_json.read_schema))

generate_json_summary = JsonSummary(config_json)


generate_json_summary.configure_model_system()
app = generate_json_summary.create_graph_agents(StateGraph(GraphState))
generate_json_summary.run_graph_agents(app,files_in_app)
logger.info(f"Plot comparision latency: {(time.time() - start):.4f} seconds")


[2024-05-22 15:26:03,271: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2024-05-22 15:26:03,273: INFO: common: yaml file: schema.yaml loaded successfully:]
[2024-05-22 15:26:03,276: INFO: common: yaml file: params.yaml loaded successfully:]
[2024-05-22 15:26:03,278: INFO: common: yaml file: models.yaml loaded successfully:]
[2024-05-22 15:26:03,283: INFO: common: yaml file: prompt_template.yaml loaded successfully:]
[2024-05-22 15:26:03,285: INFO: common: Directory already exists: artifacts/json_summary:]
[2024-05-22 15:26:03,287: INFO: 111454437: Working with model: llama3-70b-8192:]
[2024-05-22 15:26:03,798: INFO: 111454437: Agent graph created:]
[2024-05-22 15:26:03,798: INFO: 111454437: Initializing agent summary orquestration:]
[2024-05-22 15:26:03,798: INFO: 111454437: new Json summaries:]
[2024-05-22 15:26:03,798: INFO: 111454437: ------------------------------Batch: 1------------------------------:]
[2024-05-22 15:26:03,798: INFO: 111454437: Starting summar